In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
df = pd.read_csv('/content/Untitled spreadsheet - Sheet1 (2).csv')
# Sample dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {'text': self.texts[idx], 'label': self.labels[idx]}

# Tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Sample data
texts = df["Questions"]

labels = df["Difficulty"]  # Replace with your actual difficulty scores

# Create dataset and dataloader
dataset = CustomDataset(texts, labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Training parameters
lr = 1e-5
epochs = 5

# Optimizer and loss function
optimizer = AdamW(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()
i=0
# Training loop
for epoch in range(epochs):
    print(i)
    i=i+1
    for batch in dataloader:
        inputs = tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True)
        labels = torch.tensor(batch['label']).unsqueeze(1).float()

        outputs = model(**inputs)
        predictions = outputs.logits
        loss = loss_fn(predictions, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Testing the model
with torch.no_grad():
    test_text = "What is the significance of the papyrus plant in ancient Egyptian civilization?"
    test_input = tokenizer(test_text, return_tensors='pt', padding=True, truncation=True)
    test_output = model(**test_input).logits.item()
    print(f'Predicted difficulty for the test text: {test_output}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-10-a3f4c3a0a557>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).unsqueeze(1).float()


0
1
2
3
4
Predicted difficulty for the test text: 0.6074645519256592


In [11]:
with torch.no_grad():
    test_text = "If the economic practices of the Indus Valley Civilization were still being used today, what will be the consequences?"
    test_input = tokenizer(test_text, return_tensors='pt', padding=True, truncation=True)
    test_output = model(**test_input).logits.item()
    print(f'Predicted difficulty for the test text: {test_output}')

Predicted difficulty for the test text: 0.8284471035003662


In [16]:
import torch

# Save the model
model_path = "history_difficulty_model.pth"
# Save the model architecture
torch.save(model, 'history_difficulty_model_architecture.pth')


# Load the model
loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
loaded_model.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
loaded_model.eval()

# Now, you can use the loaded_model for inference
with torch.no_grad():
    test_text = "What is the significance of the papyrus plant in ancient Egyptian civilization?"
    test_input = tokenizer(test_text, return_tensors='pt', padding=True, truncation=True)
    test_output = loaded_model(**test_input).logits.item()
    print(f'Predicted difficulty for the test text: {test_output}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted difficulty for the test text: 0.6074645519256592


In [17]:
from google.colab import files

# Download the model state dictionary file
files.download('history_difficulty_model.pth')

# Download the model architecture file
files.download("history_difficulty_model_architecture.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
with torch.no_grad():
    test_text = "State life in the Middle Ages"
    test_input = tokenizer(test_text, return_tensors='pt', padding=True, truncation=True)
    test_output = loaded_model(**test_input).logits.item()
    print(f'Predicted difficulty for the test text: {test_output}')

Predicted difficulty for the test text: 0.2960759699344635
